In [3]:
import torch
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
from torch import nn
import os
import numpy as np
import random
from dataset import IPNHandDataset, zeropad_collate_fn
from lstm_model import SequenceClassification

In [2]:
# setting seed
def settle_seed(seed= 66):
  os.environ['PYTHONHASHSEED'] = str(seed)
  np.random.seed(seed)
  random.seed(seed)
  torch.manual_seed(seed) 
settle_seed()
run_name = 'normal'
summary_writer = SummaryWriter(f'log/{run_name}')

In [3]:
origin_dataset = IPNHandDataset('./data/IPN_Hand/ipn_hand.3.csv')
train_dataset, valid_dataset = torch.utils.data.random_split(origin_dataset,[1300, 289])

train_data_loader = DataLoader(train_dataset, batch_size= 32, shuffle= True, collate_fn=zeropad_collate_fn)
valid_data_loader = DataLoader(valid_dataset, batch_size= 32, shuffle= True, collate_fn=zeropad_collate_fn)

In [5]:
model = SequenceClassification()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr = 0.001,weight_decay= 1e-4)
epochs = 100
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [6]:
def evaluate(step):

    model.eval()
    valid_loss = []
    valid_acc = []
    for eval_batch in valid_data_loader:
        X, label = eval_batch
        X = X.to(device)
        label = label.to(device)
        pred = model.forward(X)
        loss = criterion(pred,label)
        acc = sum(torch.argmax(pred,dim = 1) == label) / len(label)
        valid_loss.append(loss)
        valid_acc.append(acc)
    valid_loss = sum(valid_loss) / len(valid_loss) 
    valid_acc = sum(valid_acc) / len(valid_acc)
    # log
    summary_writer.add_scalar('Loss/validate', valid_loss, step)
    summary_writer.add_scalar('Acc/validate', valid_acc, step)
    return valid_loss,valid_acc
def train():
    step = 0 
    model.to(device) 
    min_loss = float('inf') 
    for epoch in range(epochs):
        train_loss = [] 
        train_acc = []
        for batch in train_data_loader:
            model.train() 
            optimizer.zero_grad() 
            X, label = batch 
            # X = X.unsqueeze(1) # in channels 
            X = X.to(device) 
            label = label.to(device)
            pred = model.forward(X) 
            loss = criterion(pred,label) 
            acc = sum(torch.argmax(pred, dim = 1) == label) / len(label) 
            train_loss.append(loss) 
            train_acc.append(acc)
            # log
            summary_writer.add_scalar('Loss/train', loss, step)
            summary_writer.add_scalar('Acc/train', acc, step)
            loss.backward()
            optimizer.step()
            step += 1
            if step % 200 == 0:
                valid_loss,valid_acc = evaluate(step)
                print('Validing: step:%d,mean_loss:%.5f,mean_acc:%.5f' % ( step,valid_loss,valid_acc)) 
                if valid_loss < min_loss:
                    min_loss = valid_loss 
                    torch.save(model.state_dict(), './model/ipn_model_new_73.pt') 
        mean_loss = sum(train_loss) / len(train_loss) 
        mean_acc = sum(train_acc) / len(train_acc) 
        print('Trainging: epoch:%d,step:%d,mean_loss:%.5f,mean_acc:%.5f' % ( epoch,step,mean_loss,mean_acc))
train()

Trainging: epoch:0,step:41,mean_loss:0.92590,mean_acc:0.69695
Trainging: epoch:1,step:82,mean_loss:0.43739,mean_acc:0.86555
Trainging: epoch:2,step:123,mean_loss:0.28725,mean_acc:0.90457
Trainging: epoch:3,step:164,mean_loss:0.23320,mean_acc:0.92668
Validing: step:200,mean_loss:0.23143,mean_acc:0.92813
Trainging: epoch:4,step:205,mean_loss:0.17375,mean_acc:0.94954
Trainging: epoch:5,step:246,mean_loss:0.13490,mean_acc:0.96220
Trainging: epoch:6,step:287,mean_loss:0.11977,mean_acc:0.96509
Trainging: epoch:7,step:328,mean_loss:0.08699,mean_acc:0.97256
Trainging: epoch:8,step:369,mean_loss:0.09170,mean_acc:0.96677
Validing: step:400,mean_loss:0.27331,mean_acc:0.92813
Trainging: epoch:9,step:410,mean_loss:0.07713,mean_acc:0.97485
Trainging: epoch:10,step:451,mean_loss:0.05315,mean_acc:0.98430
Trainging: epoch:11,step:492,mean_loss:0.04078,mean_acc:0.98887
Trainging: epoch:12,step:533,mean_loss:0.04067,mean_acc:0.99085
Trainging: epoch:13,step:574,mean_loss:0.02355,mean_acc:0.99162
Validing

In [4]:
# load tensorboard extension
%load_ext tensorboard

In [ ]:
# launch tensorboard
# if you are using vscode, use the bottom left "..." button to change the presentation to html
%tensorboard --logdir=log